# Simple Imitation Learning in MineRL
This tutorial contains a simple example of how to build a imitation-learning based agent that can solve the MineRLNavigateDense-v0 environment. For more information about that environment, see this [MineRL Docs](http://minerl.io/docs/environments/index.html#minerlnavigatedense-v0).

For more Imitation Learning algorithms, like a Dagger in Tensorflow, see that Github repo, [Dagger](https://github.com/zsdonghao/Imitation-Learning-Dagger-Torcs).

Parts of this tutorial are based on code by [Arthur Juliani](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724).

In [1]:
from __future__ import division

import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
%matplotlib inline
import matplotlib.pyplot as plt
import math

try:
    xrange = xrange
except:
    xrange = range
    
#env_name = 'MineRLNavigateDense-v0'
env_name = 'MineRLTreechop-v0'
data_path = '/media/kimbring2/6224AA7924AA5039/minerl_data'

/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/pyth

# Setting up our Neural Network agent
This time we will be using a Convolutional Neural Network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a Jump and Camera movement. To learn more about this network, see [Convolutional Neural Networks for Visual Recognition Course](http://cs231n.stanford.edu/).

In [2]:
H = 1024

tf.reset_default_graph()

if (env_name == 'MineRLTreechop-v0'):
    state = tf.placeholder(shape=[None,64,64,3], dtype=tf.float32)
elif (env_name == 'MineRLNavigateDense-v0'):
    state = tf.placeholder(shape=[None,64,64,4], dtype=tf.float32)
    
conv1 = slim.conv2d( \
            inputs=state,num_outputs=32,kernel_size=[8,8],stride=[4,4],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)
conv2 = slim.conv2d( \
            inputs=conv1,num_outputs=64,kernel_size=[4,4],stride=[2,2],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)
conv3 = slim.conv2d( \
            inputs=conv2,num_outputs=64,kernel_size=[3,3],stride=[1,1],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)

convFlat = slim.flatten(conv3)
print("convFlat: " + str(convFlat))

if (env_name == 'MineRLTreechop-v0'):
    W = tf.get_variable("W", shape=[H,4],
               initializer=tf.contrib.layers.xavier_initializer())
    score = tf.matmul(convFlat, W)
    probability = tf.nn.softmax(score)
    real_action = tf.placeholder(shape=[None,4], dtype=tf.int32)
elif (env_name == 'MineRLNavigateDense-v0'):
    W = tf.get_variable("W", shape=[H,4],
               initializer=tf.contrib.layers.xavier_initializer())
    score = tf.matmul(convFlat, W)
    probability = tf.nn.softmax(score)
    real_action = tf.placeholder(shape=[None,4], dtype=tf.int32)
    
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=real_action, 
                                                                     logits=score))
tf.summary.scalar('loss', loss)
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

merged = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
convFlat: Tensor("Flatten/flatten/Reshape:0", shape=(?, 1024), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Train

In [ ]:
import minerl
data = minerl.data.make(env_name, data_path)

init = tf.global_variables_initializer()
with tf.Session() as sess:
# Launch the graph
    rendering = False
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    train_writer = tf.summary.FileWriter('/home/kimbring2/MineRL/train_summary/' + env_name, sess.graph)
    
    episode_count = 0
    for current_state, action, reward, next_state, done in data.sarsd_iter(num_epochs=500, max_sequence_len=200):
        #print("current_state['pov'].shape: " + str(current_state['pov'].shape))
        #print("action: " + str(action))
        #print("reward: " + str(reward))
        length = (current_state['pov'].shape)[0]

        #print("state_concat.shape: " + str(state_concat.shape))

        action_list = []
        states_list = []
        for i in range(0, length):
            if (env_name == 'MineRLTreechop-v0'):
                state_concat = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
            elif (env_name == 'MineRLNavigateDense-v0'):
                pov = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
                compass = current_state['compassAngle'][i]

                compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
                compass_channel /= 180.0
        
                state_concat = np.concatenate([pov, compass_channel], axis=-1)
            
            #print("state_concat.shape: " + str(state_concat.shape))
            #print("action['camera'][i]: " + str(action['camera'][i]))
            #print("action['camera'][i][1]: " + str(action['camera'][i][1]))
            #print("")
            '''
            (noop) : 'MineRLNavigateDense-v0'
            {'jump': 0, 'camera': [0, 0]} = 3
            {'jump': 1, 'camera': [0, 0]} = 4
            {'jump': 0, 'camera': [0, -10]} = 1
            {'jump': 0, 'camera': [0, 10]} = 2
            
            (noop) : 'MineRLTreechop-v0'
            {'forward': 0, 'jump': 0, 'camera': [0, 0]} = 3
            {'forward': 1, 'jump': 1, 'camera': [0, 0]} = 4
            {'forward': 1, 'jump': 0, 'camera': [0, -10]} = 1
            {'forward': 1, 'jump': 0, 'camera': [0, 10]} = 2
            '''
            
            if (env_name == 'MineRLNavigateDense-v0'):
                if (action['camera'][i][1] < 0):
                    action_ = [1, 0, 0, 0]
                elif (action['camera'][i][1] > 0):
                    action_ = [0, 1, 0, 0]
                else:
                    if (action['jump'][i] == 0):
                        action_ = [0, 0, 1, 0]
                    else:
                        action_ = [0, 0, 0, 1]
            elif (env_name == 'MineRLTreechop-v0'):
                if (action['camera'][i][1] < 0):
                    action_ = [1, 0, 0, 0]
                elif (action['camera'][i][1] > 0):
                    action_ = [0, 1, 0, 0]
                else:
                    if ( (action['jump'][i] == 0) & (action['forward'][i] == 0) ):
                        action_ = [0, 0, 1, 0]
                    elif ( (action['jump'][i] == 1) & (action['forward'][i] == 1) ):
                        action_ = [0, 0, 0, 1]
            
            states_list.append(state_concat)
            action_list.append(action_)
        
        episode_count = episode_count + 1
        
        feed_dict = {state:np.stack(states_list, 0),
                     real_action:np.stack(action_list, 0)
                    }
    
        if episode_count % 100 == 0:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict)
            train_writer.add_summary(summary, episode_count)

        sess.run(train_step, feed_dict=feed_dict)
        
        if episode_count % 100 == 0:
            model_path = '/home/kimbring2/MineRL/model/' + env_name
            saver.save(sess, model_path + '/model-' + str(episode_count) + '.cptk')
            print("Saved Model")

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

# Test

In [3]:
import minerl
import gym

env = gym.make(env_name)
obs = env.reset()

init = tf.global_variables_initializer()
with tf.Session() as sess:
# Launch the graph
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    
    print('Loading Model...') 
    path = '/home/kimbring2/MineRL/model/' + env_name
    ckpt = tf.train.get_checkpoint_state(path)
    saver.restore(sess, ckpt.model_checkpoint_path)
    
    env.init()
    obs = env.reset()
    net_reward = 0
    while True:
        if (env_name == 'MineRLTreechop-v0'):
            state_concat = obs['pov'].astype(np.float32) / 255.0 - 0.5
        elif (env_name == 'MineRLNavigateDense-v0'):
            pov = obs['pov'].astype(np.float32) / 255.0 - 0.5
            compass = obs['compassAngle']
            compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
            compass_channel /= 180.0
            state_concat = np.concatenate([pov, compass_channel], axis=-1)
            
        action_probability = sess.run(probability, feed_dict={state:[state_concat]})

        '''
            (noop) : 'MineRLNavigateDense-v0'
            {'jump': 0, 'camera': [0, 0]} = 3
            {'jump': 1, 'camera': [0, 0]} = 4
            {'jump': 0, 'camera': [0, -10]} = 1
            {'jump': 0, 'camera': [0, 10]} = 2
            
            (noop) : 'MineRLTreechop-v0'
            {'forward': 0, 'jump': 0, 'camera': [0, 0]} = 3
            {'forward': 1, 'jump': 1, 'camera': [0, 0]} = 4
            {'forward': 1, 'jump': 0, 'camera': [0, -10]} = 1
            {'forward': 1, 'jump': 0, 'camera': [0, 10]} = 2
         '''
        
        action = env.action_space.noop()
        if (env_name == 'MineRLNavigateDense-v0'):
            if (np.argmax(action_probability) == 0):
                action['camera'] = [0, -10]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            elif (np.argmax(action_probability) == 1):
                action['camera'] = [0, 10]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            elif (np.argmax(action_probability) == 2):
                action['camera'] = [0, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            else:
                action['camera'] = [0, 0]
                action['jump'] = 1
                action['forward'] = 1
                action['sprint'] = 1
        elif (env_name == 'MineRLTreechop-v0'):
            if (np.argmax(action_probability) == 0):
                action['camera'] = [0, -10]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (np.argmax(action_probability) == 1):
                action['camera'] = [0, 10]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (np.argmax(action_probability) == 2):
                action['camera'] = [0, 0]
                action['jump'] = 0
                action['forward'] = 0
                action['attack'] = 1
                action['sprint'] = 0
            else:
                action['camera'] = [0, 0]
                action['jump'] = 1
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
        
        action['back'] = 0
        action['left'] = 0
        action['right'] = 0

        obs, reward, done, info = env.step(action)
        
        #if done == True:
        #    break

        net_reward += reward
        print("Total reward: ", net_reward)

Loading Model...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/kimbring2/MineRL/model/MineRLTreechop-v0/model-313500.cptk
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total reward:  0.0
Total rewa

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

...

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward:  1.0
Total reward

KeyboardInterrupt: 